In [20]:
import numpy as np
import matplotlib.pyplot as plt
from schmidt_spike_removal import schmidt_spike_removal
from homomorphic_envelope_with_hilbert import homomorphic_envelope_with_hilbert
from get_PSD_feature_Springer_HMM import get_PSD_feature_Springer_HMM
from getDWT import getDWT
from scipy.signal import butter, filtfilt, hilbert, resample_poly, hilbert

In [3]:
include_wavelet = 0
featuresFs = 50
fs = 1000  # original sampling frequency
audio_data = np.loadtxt('train/train_recording_1.txt')

In [12]:
# 25-400Hz 4th order Butterworth band pass
lowcut = 25.0
highcut = 400.0
order = 4
b, a = butter(order, [lowcut, highcut], btype='bandpass', fs=fs)
audio_data = filtfilt(b, a, audio_data)

In [ ]:
# Spike removal from the original paper:
audio_data = schmidt_spike_removal(audio_data, fs)

In [19]:
# Find the homomorphic envelope
homomorphic_envelope = homomorphic_envelope_with_hilbert(audio_data, fs)
downsampled_homomorphic_envelope = resample_poly(homomorphic_envelope, featuresFs, fs)
# normalize downsampled homomorphic envelope to mean 0 and std 1
downsampled_homomorphic_envelope = (downsampled_homomorphic_envelope - np.mean(downsampled_homomorphic_envelope)) / np.std(downsampled_homomorphic_envelope)

In [21]:
# Hilbert Envelope
hilbert_envelope = np.abs(hilbert(audio_data))
downsampled_hilbert_envelope = resample_poly(hilbert_envelope, featuresFs, fs)
# normalize downsampled hilbert envelope to mean 0 and std 1
downsampled_hilbert_envelope = (downsampled_hilbert_envelope - np.mean(downsampled_hilbert_envelope)) / np.std(downsampled_hilbert_envelope)

In [23]:
# Power spectral density feature
psd = get_PSD_feature_Springer_HMM(audio_data, fs, frequency_limit_low=40, frequency_limit_high=60)
psd = resample_poly(psd, len(downsampled_homomorphic_envelope), len(psd))
psd = (psd - np.mean(psd)) / np.std(psd)

In [28]:
# Wavelet features:

if(include_wavelet):
    wavelet_level = 3
    wavelet_name ='rbio3.9'
    
    # Audio needs to be longer than 1 second for getDWT to work:
    if(len(audio_data)< fs*1.025):
        audio_data = np.concatenate[audio_data, zeros(round(0.025*fs),1)]
    
    [cD, cA] = getDWT(audio_data,wavelet_level,wavelet_name)
    
    wavelet_feature = np.abs(cD[wavelet_level-1,:])
    wavelet_feature = wavelet_feature[1:len(homomorphic_envelope)]
    downsampled_wavelet = resample_poly(wavelet_feature, featuresFs, fs)
    downsampled_wavelet =  (downsampled_wavelet - np.mean(downsampled_wavelet)) / np.std(downsampled_wavelet)

if(include_wavelet):
    PCG_Features = [downsampled_homomorphic_envelope, downsampled_hilbert_envelope, psd, downsampled_wavelet]
else:
    PCG_Features = [downsampled_homomorphic_envelope, downsampled_hilbert_envelope, psd]

In [29]:
if(include_wavelet):
    PCG_Features = np.column_stack([downsampled_homomorphic_envelope, downsampled_hilbert_envelope, psd, downsampled_wavelet])
else:
    PCG_Features = np.column_stack([downsampled_homomorphic_envelope, downsampled_hilbert_envelope, psd])